# Importing the neccessary Libraries

In [1]:
import nltk
from nltk.stem import WordNetLemmatizer
lemmatizer = WordNetLemmatizer()
import pickle
import numpy as np
from keras.models import load_model
model = load_model('chatbot_model.h5')
import json
import random

2022-05-22 14:49:14.915051: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2022-05-22 14:49:14.915190: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.
2022-05-22 14:49:46.729624: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcuda.so.1'; dlerror: libcuda.so.1: cannot open shared object file: No such file or directory
2022-05-22 14:49:46.729719: W tensorflow/stream_executor/cuda/cuda_driver.cc:269] failed call to cuInit: UNKNOWN ERROR (303)
2022-05-22 14:49:46.729790: I tensorflow/stream_executor/cuda/cuda_diagnostics.cc:156] kernel driver does not appear to be running on this host (sanam0876-HP-Laptop-15-da0xxx): /proc/driver/nvidia/version does not exist
2022-05-22 14:49:46.774119: I tensorflow/core/p

# Load the required files

In [2]:
intents = json.loads(open('intents.json').read())
words = pickle.load(open('words.pkl','rb'))         #mode = r: read and b: binary stream
classes = pickle.load(open('classes.pkl','rb'))

In [27]:
print(words)

["'s", ',', 'a', 'adverse', 'all', 'anyone', 'are', 'awesome', 'be', 'behavior', 'blood', 'by', 'bye', 'can', 'causing', 'chatting', 'check', 'could', 'data', 'day', 'detail', 'do', 'dont', 'drug', 'entry', 'find', 'for', 'give', 'good', 'goodbye', 'have', 'hello', 'help', 'helpful', 'helping', 'hey', 'hi', 'history', 'hola', 'hospital', 'how', 'i', 'id', 'is', 'later', 'list', 'load', 'locate', 'log', 'looking', 'lookup', 'management', 'me', 'module', 'nearby', 'next', 'nice', 'of', 'offered', 'open', 'patient', 'pharmacy', 'pressure', 'provide', 'reaction', 'related', 'result', 'search', 'searching', 'see', 'show', 'suitable', 'support', 'task', 'thank', 'thanks', 'that', 'there', 'till', 'time', 'to', 'transfer', 'up', 'want', 'what', 'which', 'with', 'you']


In [28]:
print(intents)

{'intents': [{'tag': 'greeting', 'patterns': ['Hi there', 'How are you', 'Is anyone there?', 'Hey', 'Hola', 'Hello', 'Good day'], 'responses': ['Hello, thanks for asking', 'Good to see you again', 'Hi there, how can I help?'], 'context': ['']}, {'tag': 'goodbye', 'patterns': ['Bye', 'See you later', 'Goodbye', 'Nice chatting to you, bye', 'Till next time'], 'responses': ['See you!', 'Have a nice day', 'Bye! Come back again soon.'], 'context': ['']}, {'tag': 'thanks', 'patterns': ['Thanks', 'Thank you', "That's helpful", 'Awesome, thanks', 'Thanks for helping me'], 'responses': ['Happy to help!', 'Any time!', 'My pleasure'], 'context': ['']}, {'tag': 'noanswer', 'patterns': [], 'responses': ["Sorry, can't understand you", 'Please give me more info', 'Not sure I understand'], 'context': ['']}, {'tag': 'options', 'patterns': ['How you could help me?', 'What you can do?', 'What help you provide?', 'How you can be helpful?', 'What support is offered'], 'responses': ['I can guide you through

In [29]:
print(classes)

['adverse_drug', 'blood_pressure', 'blood_pressure_search', 'goodbye', 'greeting', 'hospital_search', 'options', 'pharmacy_search', 'thanks']


# Defining functions

In [3]:
#perform pre-processing operations on the user input sentence

def clean_up_sentence(sentence,show_details=True):
    
    # tokenize the pattern - split words into array
    sentence_words = nltk.word_tokenize(sentence)
    
    if(show_details):
        print("After Tokonize--->",sentence_words)
        
    # stem each word - create short form for word
    sentence_words = [lemmatizer.lemmatize(word.lower()) for word in sentence_words]
    return sentence_words

sentence = "I am suffering from corona, Whats my daily routine should be?" #Example Sentence
clean_up_sentence(sentence,show_details=True) #Calling Function

After tokonize---> ['I', 'am', 'suffering', 'from', 'corona', ',', 'Whats', 'my', 'daily', 'routine', 'should', 'be', '?']


['i',
 'am',
 'suffering',
 'from',
 'corona',
 ',',
 'whats',
 'my',
 'daily',
 'routine',
 'should',
 'be',
 '?']

In [4]:
#The bag-of-words model is a simplifying representation used in natural language processing and information retrieval.
#Defining a function bow() to get the bag of words
def bow(sentence, words, show_details=True):
    
    # tokenize the pattern by calling the function
    sentence_words = clean_up_sentence(sentence,show_details=False)
    
    # bag of words - matrix of N words, vocabulary matrix
    bag = [0]*len(words)   #bag = [0,0,0,0,0,0,0,0,....]
    
    for s in sentence_words:    #s for each word in sentence words
        for i,w in enumerate(words):  #i for index and w for each word in words
            if w == s:
                
                # assign 1 if current word is in the vocabulary position
                bag[i] = 1
                
                if show_details:
                    print(w)
                    print(s)
                    print ("found in bag: %s" % w)
    return(np.array(bag))

bow(sentence, words, show_details=True) #calling the function

i
i
found in bag: i
,
,
found in bag: ,
be
be
found in bag: be


array([0, 1, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0])

In [8]:
#Defining function to predict the class of the sentence using the model created in training_bot 

def predict_class(sentence, model):
    
    # filter out predictions below a threshold
    p = bow(sentence, words,show_details=False) #calling the function
    
    #predict will give the relevance accuracy for the words found in the bag with predefined classes 
    res = model.predict(np.array([p]))[0]   #[0] to convert the array into the list                                               
    print("res -->",res)
    
    ERROR_THRESHOLD = 0.25 #Minimum value of the accuracy shoukd to be 0.25 or 25%
    
    results = [[i,r] for i,r in enumerate(res) if r>ERROR_THRESHOLD] #filter out the values which are below 0.25
    print("results-->",results)
    
    # sort by strength of probability
    results.sort(key=lambda x: x[1], reverse=True) #lambda: inline function  reverse: descending order
    
    #x = [0:index,1:probability] here we need to sort the list according to the probability value thats why x[1]
    
    #def function(x):
         #return x[1]
    print("results sorted -->",results)
    
    return_list = []  #empty list
    
    for r in results:
        return_list.append({"intent": classes[r[0]], "probability": str(r[1])})  
    return return_list

predict_class(sentence, model) #calling the function

res --> [7.8403385e-04 1.3344385e-02 3.0120698e-04 5.9673208e-01 1.6238218e-02
 3.5895498e-03 3.6674005e-01 9.3521824e-04 1.3352295e-03]
results--> [[3, 0.5967321], [6, 0.36674005]]
results sorted --> [[3, 0.5967321], [6, 0.36674005]]


[{'intent': 'goodbye', 'probability': '0.5967321'},
 {'intent': 'options', 'probability': '0.36674005'}]

In [9]:
#Defining the function to get the response from the intents.json file according the predicted class

def getResponse(ints, intents_json):
    tag = ints[0]['intent']   #ints = result from the predict_class() 
                              #Here we will the predicted intent to the tag
        
    list_of_intents = intents_json['intents']
    
    for i in list_of_intents:
        if(i['tag']== tag):
            result = random.choice(i['responses']) #randomly choose a response from the predicted class
            print(result)
            break
    return result

In [10]:
#Defining the main function that will be use in the graphical output
def chatbot_response(msg):
    ints = predict_class(msg, model)
    res = getResponse(ints, intents)
    return res

# Build a GUI for the Bot using Tkinter

In [ ]:
#Creating GUI with tkinter #Tkinter provides a fast and easy way to create GUI applications.
import tkinter
from tkinter import * # * means all the modules of the tkinter

#Defining function for the sending activity
def send():
    #EntryBox is the class for creating the box for the message
    
    msg = EntryBox.get("1.0",'end-1c').strip()  
    #"1.0" means that the input should be read from line one, character zero (ie: the very first character)
    #The end-1c part means to read until the end of the text box is reached
    
    EntryBox.delete("0.0",END)
    #Reset the the message box for further use

    if msg != '':
        
        #ChatLog: Class for creating the main chat window
        ChatLog.config(state=NORMAL) #Enable or active the Chat Window
        
        ChatLog.insert(END, "You: " + msg + '\n\n')
        
        ChatLog.config(foreground="#442265", font=("Verdana", 12 )) #Set the foreground color and font of the text
    
        res = chatbot_response(msg) #Calling the function with the user input message
        
        ChatLog.insert(END, "Bot: " + res + '\n\n')
            
        ChatLog.config(state=DISABLED) #Disable or deactive the Chat Window after the chat ended
        ChatLog.yview(END) #yview: show the conversaation in vertical view
 

base = Tk() #Tk() class create an instance of tkinter frame (The graphical window)
base.title("Hello")
base.geometry("400x500") #Set the width and height of the frame
base.resizable(width=FALSE, height=FALSE) #Size cant be changed

#Create Chat window
ChatLog = Text(base, bd=0, bg="white", height="8", width="50", font="Arial",)
#bd = border width

ChatLog.config(state=DISABLED) #Chat Window Will be disabled by deafult

#Bind scrollbar to Chat window
scrollbar = Scrollbar(base, command=ChatLog.yview, cursor="heart")
ChatLog['yscrollcommand'] = scrollbar.set #set the scrolllbar to the chat window

#Create Button to send message
SendButton = Button(base, font=("Verdana",12,'bold'), text="Send", width="12", height=5,
                    bd=0, bg="#32de97", activebackground="#3c9d9b",fg='#ffffff',
                    command= send )

#Create the box to enter message
EntryBox = Text(base, bd=0, bg="white",width="29", height="5", font="Arial")
#EntryBox.bind("<Return>", send)


#Place all components on the screen
scrollbar.place(x=376,y=6, height=386)
ChatLog.place(x=6,y=6, height=386, width=370)
EntryBox.place(x=128, y=401, height=90, width=265)
SendButton.place(x=6, y=401, height=90)

base.mainloop() 
#This method listens for events, such as button clicks and keeps the frame running until you close the window.

res --> [1.0619105e-05 3.8147980e-05 3.7042987e-06 6.2468091e-05 9.9970740e-01
 1.2276216e-05 1.7246060e-05 2.9140303e-06 1.4512692e-04]
results--> [[4, 0.9997074]]
results sorted --> [[4, 0.9997074]]
Hello, thanks for asking
res --> [1.4705162e-09 1.8679746e-08 1.7051097e-09 2.2721487e-08 2.7954077e-09
 3.7790951e-08 9.9999988e-01 6.3983915e-11 2.6144964e-10]
results--> [[6, 0.9999999]]
results sorted --> [[6, 0.9999999]]
Offering support for Adverse drug reaction, Blood pressure, Hospitals and Pharmacies
